In [2]:
import numpy as np
from Data_Savior_J import load_file

Xz = load_file("./classifier_data/walk1.data")
Xz = np.vstack((Xz,load_file("./classifier_data/walk1U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk1D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk2.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk2U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk2D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk3.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk3U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk3D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk4.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk4U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk4D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk5.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk5U.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk5D.data")))

## Vetor de features para classificação:

$X_c = [a~| ~av~| ~aa~| ~l\_a~| ~l\_av~| ~l\_aa~| ~pos\_foot\_r~| ~pos\_foot\_l~| ~vz\_r~| ~vz\_l~| ~C]$
$X_c = [0~|~~1~~| ~~2~~| ~~~3~~| ~~~4~~~| ~~~~5~~| ~~~~~~~6~~~~~~~~~~| ~~~~~~~~7~~~~~~~~| ~~~8~~~| ~~~~9~~| 10]$

#### $a \rightarrow$ ângulo do joelho direito; $av \rightarrow$ velocidade angular do joelho direito; $aa \rightarrow$ aceleração angular do joelho direito;
#### $a \rightarrow$ ângulo do joelho esquerdo; $av \rightarrow$ velocidade angular do joelho esquerdo; $aa \rightarrow$ aceleração angular do joelho esquerdo; 
#### $pos\_foot\_r \rightarrow$ posição do tornozelo direito em relação ao sacro; $pos\_foot\_l \rightarrow$ posição do tornozelo esquerdo em relação ao sacro;
#### $vz\_r \rightarrow$ velocidade do trocanter direito no eixo z; $vz\_l \rightarrow$ velocidade do trocanter esquerdo no eixo z;
#### $C \rightarrow$ indice de classificação
## Indice de classificação $"c"$:
#### $C = 0 \rightarrow$ Marcha normal;
#### $C = 1 \rightarrow$ Marcha de subida de escada;
#### $C = 2 \rightarrow$ Marvha de descidade escada.

In [3]:
import numpy as np

X = Xz[:,[0,1,2,3,4,5,6,7,8,9]] #Utilizando as features distância entre tornozelos direto/esquerdo e sacro
#X = Xz[:,[0,1,2,3,4,5]]

yz = Xz[:,[10]]
y = np.array([])
for i in range(len(yz)):
    y = np.hstack((y,yz[i]))

X.shape, y.shape

((817, 10), (817,))

In [4]:
np.unique(y) # possíveis valores de y

array([ 0.,  1.,  2.])

In [123]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, 
                                                    random_state=10)

In [124]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)


In [125]:
print X_train_std.shape, X_test.shape

(612, 10) (205, 10)


In [126]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets

print 'Processing time RBF'
%time rbf_svc = svm.SVC(kernel='rbf', gamma=0.2, C=5, decision_function_shape='ovr').fit(X_train_std, y_train)
print ''
print 'Processing time Polynomial'
#%time poly_svc = svm.SVC(kernel='poly', degree=4, coef0=6.0, C=0.5, decision_function_shape='ovr').fit(X_train_std, y_train)
%time poly_svc = svm.SVC(kernel='poly', degree=2, coef0=4.7, C=48.9, decision_function_shape='ovr').fit(X_train_std, y_train)

Processing time RBF
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 9.21 ms

Processing time Polynomial
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 10.7 ms


In [127]:
def run_svm(svc, X_test_std, y_test):
    y_pred = svc.predict(X_test_std)
    from sklearn.metrics import accuracy_score
    if (svc==rbf_svc):
        print ('SVM-RBF accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))
    elif(svc==poly_svc):
        print ('SVM-Polynomial accuracy:--->%.2f %%' % (accuracy_score(y_test, y_pred)*100))

In [128]:
run_svm(rbf_svc, X_test_std, y_test)
run_svm(poly_svc, X_test_std, y_test)

SVM-RBF accuracy:---------->99.51 %
SVM-Polynomial accuracy:--->99.02 %


In [20]:
from sklearn.grid_search import GridSearchCV
param_grid = [
    {
        'C'     : [0.001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 3, 6, 8,10, 48.5, 48.56, 48.8, 49, 49.01, 39.1, 49.2, 50, 70, 80,
                   100, 1000], 
        'degree' : [1, 2, 3, 4, 5, 6, 7, 8],
        'coef0' : [1, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0], 
        'random_state' : [1, 2, 3, 4, 5, 6, 10, 20, 40, 100, 1000],
        'kernel': ['poly']
    },
]    
clf = GridSearchCV(svm.SVC(C=1), param_grid, cv=5)
%time clf.fit(X_train_std, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
for params, mean_score, scores in clf.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r"
          % (mean_score, scores.std() * 2, params))
print()


CPU times: user 21min 51s, sys: 328 ms, total: 21min 51s
Wall time: 21min 51s
Best parameters set found on development set:
()
{'kernel': 'poly', 'C': 0.01, 'random_state': 1, 'coef0': 3.5, 'degree': 6}
()
Grid scores on development set:
()
0.348 (+/-0.003) for {'kernel': 'poly', 'C': 0.001, 'random_state': 1, 'coef0': 1, 'degree': 1}
0.348 (+/-0.003) for {'kernel': 'poly', 'C': 0.001, 'random_state': 2, 'coef0': 1, 'degree': 1}
0.348 (+/-0.003) for {'kernel': 'poly', 'C': 0.001, 'random_state': 3, 'coef0': 1, 'degree': 1}
0.348 (+/-0.003) for {'kernel': 'poly', 'C': 0.001, 'random_state': 4, 'coef0': 1, 'degree': 1}
0.348 (+/-0.003) for {'kernel': 'poly', 'C': 0.001, 'random_state': 5, 'coef0': 1, 'degree': 1}
0.348 (+/-0.003) for {'kernel': 'poly', 'C': 0.001, 'random_state': 6, 'coef0': 1, 'degree': 1}
0.348 (+/-0.003) for {'kernel': 'poly', 'C': 0.001, 'random_state': 10, 'coef0': 1, 'degree': 1}
0.348 (+/-0.003) for {'kernel': 'poly', 'C': 0.001, 'random_state': 20, 'coef0': 1, 'd

In [314]:

def frange(start, stop, step):
    i = start
    while i < stop:
        yield i
        i += step
v = []
for i in frange(1, 8, 0.5):
     v.append(i)
%time v


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.81 µs


[1, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5]

In [21]:
%time
def frange(start, stop, step):
    i = start
    while i < stop:
        yield i
        i += step
g = []
for i in frange(0.5, 0.56, 0.0005):
     g.append(i)
g

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.05 µs


[5,
 5.05,
 5.1,
 5.1499999999999995,
 5.199999999999999,
 5.249999999999999,
 5.299999999999999,
 5.349999999999999,
 5.399999999999999,
 5.449999999999998,
 5.499999999999998,
 5.549999999999998,
 5.599999999999998,
 5.649999999999998,
 5.6999999999999975,
 5.749999999999997,
 5.799999999999997,
 5.849999999999997,
 5.899999999999997,
 5.949999999999997,
 5.9999999999999964,
 6.049999999999996,
 6.099999999999996,
 6.149999999999996,
 6.199999999999996,
 6.249999999999996,
 6.299999999999995,
 6.349999999999995,
 6.399999999999995,
 6.449999999999995,
 6.499999999999995]